# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [34]:
# import libraries
import pandas as pd
import numpy as np
import re
from time import time
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support,accuracy_score, classification_report, label_ranking_average_precision_score

In [6]:
# load data from database
engine = create_engine('sqlite:///../data/Disaster_Response.db')
table_name = "messages_tab"
df = pd.read_sql_table(table_name, engine)
X = df.message
Y = df.drop(['id','message','genre','original'], axis=1)

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    """
    Function: tokenize text
    Args:
        text (str): string to tokenize
    Return:
        clean_tokens (list of str): list of cleaned and tokenized strings
    """
    # 1. normalization
    # remove punctuation and capitalization of words
    text = re.sub(r'[^a-zA-Z0-9]', " ", text.lower())
    
    # 2. tokenization
    # tokenize words from text
    tokens = word_tokenize(text)
    
    # 3. stop word removal
    # remove all words that can be found in the stopwords dictionary
    tokens = [w for w in tokens if w not in stopwords.words("english")]

    # 4. stemming and lemmtization
    # define lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [16]:
# check funtion
print("Message: ", X[0])
print("Tokens: ", tokenize(X[0]))

Message:  Weather update - a cold front from Cuba that could pass over Haiti
Tokens:  ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
# define ML pipeline with TF-IDF Vectorizer and MultiOutputClassifier (as estimator we use a RandomForestClassifier)
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=150)))    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
# split dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state=42)

# train pipeline
start = time()
pipeline.fit(X_train, y_train)
print("Training Time: {} seconds".format(time()-start))

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
def display_accuracy(y_test, y_pred):
    """
    Function: show accuracy metrics for predicted test dataset (f1_score, precision, recall)
    Args:
        y_test (pd.Series): original labels of test dataset
        y_pred (pd.Series): predicted labels of test dataset
    Return:
        None
    """
    for i, col in enumerate(y_test):
        print(col,':')
        print(classification_report(y_test[col], y_pred[:, i]))

In [28]:
y_pred = pipeline.predict(X_test)

In [49]:
display_accuracy(y_test, y_pred)

related :
              precision    recall  f1-score   support

           0       0.70      0.42      0.53      1245
           1       0.84      0.94      0.89      3998

   micro avg       0.82      0.82      0.82      5243
   macro avg       0.77      0.68      0.71      5243
weighted avg       0.81      0.82      0.80      5243

request :
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4352
           1       0.86      0.50      0.63       891

   micro avg       0.90      0.90      0.90      5243
   macro avg       0.88      0.74      0.79      5243
weighted avg       0.90      0.90      0.89      5243

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00      0.99   

C:\Users\Kroening\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



missing_people :
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5180
           1       1.00      0.03      0.06        63

   micro avg       0.99      0.99      0.99      5243
   macro avg       0.99      0.52      0.53      5243
weighted avg       0.99      0.99      0.98      5243

refugees :
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5073
           1       0.69      0.06      0.12       170

   micro avg       0.97      0.97      0.97      5243
   macro avg       0.83      0.53      0.55      5243
weighted avg       0.96      0.97      0.96      5243

death :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4996
           1       0.88      0.15      0.26       247

   micro avg       0.96      0.96      0.96      5243
   macro avg       0.92      0.57      0.62      5243
weighted avg       0.96      0.96    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.